In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/NLP\ project/

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1MmY0pN1b5xL_C2CijM9ImcFM-UFt4bwr/NLP project


In [2]:
#import packages
import pandas as pd
import numpy as np
import os
import sys 
from keras.models import Sequential
from keras.layers import LSTM, Activation, Flatten, Dropout, Dense, Embedding, TimeDistributed, SimpleRNN
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Data Preprocessing

In [3]:
path = "./songs_edit_revised/"
corpus = ''

all_files = os.listdir(path)
for file in all_files:
  with open(os.path.join(path, file)) as f:
    text = f.read()
    corpus+= '\n\n'+ text


In [4]:
components = corpus.split('\n\n')

In [5]:
print(len(components))

288


In [6]:
substring = "<VERSE>"
verse_list = [string.split('\n') for string in components if substring in string]

for song in verse_list:
  song.pop(0)

# new_verse_list = [item for sublist in verse_list for item in sublist]
# new_verse_list[:5]

In [7]:
substring = "<CHORUS>"
chorus_list = [string.split('\n') for string in components if substring in string]
for song in chorus_list:
  song.pop(0)

# new_chorus_list = [item for sublist in chorus_list for item in sublist]
# new_chorus_list[:5]


In [8]:
substring = "<PRECHORUS>"
pre_list = [string.split('\n') for string in components if substring in string]
for song in pre_list:
  song.pop(0)

# new_pre_list = [item for sublist in pre_list for item in sublist]
# new_pre_list[:5]



In [9]:
substring = "<BRIDGE>"
bridge_list = [string.split('\n') for string in components if substring in string]
for song in bridge_list:
  song.pop(0)

# new_bridge_list = [item for sublist in bridge_list for item in sublist]
# new_bridge_list[:5]

In [10]:
substring = "<OTHERS>"
others_list = [string.split('\n') for string in components if substring in string]
for song in others_list:
  song.pop(0)

# new_others_list = [item for sublist in others_list for item in sublist]
# new_others_list[:5]

Generating lyrics for VERSE

In [11]:
verse_df = pd.DataFrame(
    {'verse': verse_list})

verse_df['verse'] = verse_df['verse'].str.join(', ')
pd.options.display.max_colwidth = 500
verse_df.head()

,verse
0,"Will you make this island, Amazing in all ways, Surprises every corner, Delightful nights and days, Will you take this country, And turn it from a place, To a home that greets you, With smiles on every face, Will you come on this brave journey, Will you help to make it real, Will you write us grand new stories, Songs that everyone will feel"
1,"Will you take this city, And turn it from a place, To a home that greets you, With smiles on every face, Will you come on this brave journey, Will you help to make it real, Will you write us grand new stories, Songs that everyone will feel"
2,"Wake up, she said, Look it’s a beautiful day, Downstairs to the kitchen door, And then away, Into the light, Morning feeling lives on, Come the clouds, the moon, And morning is gone"
3,"Born today some years ago, And had a happy childhood, But I fell in love and out, Nothing changed, Lived a life of nothing much, But then how much can one expect?, So there you are, My life has gone, But I’m the same"
4,"Get up, he said, Hurry or you might be late, Everyday you hurry off to keep your days, Learn something new, What are you hoping to prove?, Make some money, Find a wife, Have a kid or two"


In [12]:
from google.colab import files
src = list(files.upload().values())[0]
open('utilities.py','wb').write(src)
import utilities as U

Saving utilities.py to utilities (1).py


In [13]:
"""
Remove punctuations, decontractions, emojis and URLs
"""

verse_df['verse'] = verse_df.apply(lambda row : U.remove_punct(row['verse']), axis=1)
verse_df['verse'] = verse_df.apply(lambda row : U.remove_url(row['verse']), axis=1)
verse_df['verse'] = verse_df.apply(lambda row : U.remove_emoji(row['verse']), axis=1)
verse_df['verse'] = verse_df.apply(lambda row : U.decontraction(row['verse']), axis=1)
verse_df['verse']=  verse_df.apply(lambda row : row['verse'].lower(), axis=1)
verse_df.head()

,verse
0,will you make this island amazing in all ways surprises every corner delightful nights and days will you take this country and turn it from a place to a home that greets you with smiles on every face will you come on this brave journey will you help to make it real will you write us grand new stories songs that everyone will feel
1,will you take this city and turn it from a place to a home that greets you with smiles on every face will you come on this brave journey will you help to make it real will you write us grand new stories songs that everyone will feel
2,wake up she said look its a beautiful day downstairs to the kitchen door and then away into the light morning feeling lives on come the clouds the moon and morning is gone
3,born today some years ago and had a happy childhood but i fell in love and out nothing changed lived a life of nothing much but then how much can one expect so there you are my life has gone but im the same
4,get up he said hurry or you might be late everyday you hurry off to keep your days learn something new what are you hoping to prove make some money find a wife have a kid or two


In [14]:
verses = verse_df['verse'].tolist()

In [15]:
verses[:10]

['will you make this island amazing in all ways surprises every corner delightful nights and days will you take this country and turn it from a place to a home that greets you with smiles on every face will you come on this brave journey will you help to make it real will you write us grand new stories songs that everyone will feel',
 'will you take this city and turn it from a place to a home that greets you with smiles on every face will you come on this brave journey will you help to make it real will you write us grand new stories songs that everyone will feel',
 'wake up she said look its a beautiful day downstairs to the kitchen door and then away into the light morning feeling lives on come the clouds the moon and morning is gone',
 'born today some years ago and had a happy childhood but i fell in love and out nothing changed lived a life of nothing much but then how much can one expect so there you are my life has gone but im the same',
 'get up he said hurry or you might be l

In [16]:
# Save verses in .txt file
with open('verses.txt', 'w',encoding="utf-8") as filehandle:  
    for listitem in verses:
        filehandle.write('%s\n' % listitem)

In [17]:
textFileName = 'verses.txt'
raw_text = open(textFileName, encoding = 'UTF-8').read()

Reference: https://colab.research.google.com/drive/1wlZXZBvOo93pAmTtEUeTlPsgAP4D1bLA#scrollTo=WsWUvqD0sGsX

In [18]:
# Mapping chars to ints
#Make two dictionaries: one to convert chars to ints , the other to convert ints back to chars

#create a list of sorted unique characters
chars = sorted(list(set(raw_text)))
int_chars = dict((i, c) for i, c in enumerate(chars))
chars_int = dict((c, i) for i, c in enumerate(chars))

# int_chars = dict((i, c) for i, c in enumerate(chars))
# chars_int = dict((i, c) for c, i in enumerate(chars))

In [19]:
# Get number of chars and vocab in our text :
n_chars = len(raw_text)
n_vocab = len(chars)

In [20]:
print('Total Characters : ' , n_chars) # number of all the characters in verses.txt
print('Total Vocab : ', n_vocab) # number of unique characters

Total Characters :  22309
Total Vocab :  28


In [21]:
#make samples and labels
# process the dataset:
seq_len = 100
data_X = []
data_y = []

for i in range(0, n_chars - seq_len, 1):
    # Input sequence(will be used as samples)
    seq_in  = raw_text[i:i+seq_len]
    # Output sequence (will be used as target)
    seq_out = raw_text[i + seq_len]
    # Store samples in data_X
    data_X.append([chars_int[char] for char in seq_in])
    # Store targets in data_y
    data_y.append(chars_int[seq_out])
n_patterns = len(data_X)
print( 'Total Patterns : ', n_patterns)

Total Patterns :  22209


In [22]:
"""
prepare the samples and labels to be ready to go into our model.

- Reshape the samples
- Normalize them
- One hot encode the output targets
"""

# Reshape X to be suitable to go into RNN :
X = np.reshape(data_X , (n_patterns, seq_len, 1))
# Normalizing input data :
X = X/ float(n_vocab)
# One hot encode the output targets :
y = np_utils.to_categorical(data_y)

Build and Train RNN Model

In [23]:
model = Sequential()

#add output layer
model.add(SimpleRNN(units=128, input_shape=(X.shape[1], X.shape[2]), activation="relu"))
model.add(Dense(y.shape[1]))
model.add(Dropout(0.2, input_shape=(X.shape[1], X.shape[2])))
model.add(Flatten())
model.add(Dense(y.shape[1]))
model.add(Activation('softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 128)               16640     
                                                                 
 dense (Dense)               (None, 28)                3612      
                                                                 
 dropout (Dropout)           (None, 28)                0         
                                                                 
 flatten (Flatten)           (None, 28)                0         
                                                                 
 dense_1 (Dense)             (None, 28)                812       
                                                                 
 activation (Activation)     (None, 28)                0         
                                                                 
Total params: 21,064
Trainable params: 21,064
Non-traina

In [25]:
# Fit the model :
model_params = {'epochs':20,
                'batch_size':32,
                #'callbacks':callbacks_list,
                'verbose':1,
                'validation_split':0.2,
                'validation_data':None,
                'shuffle': True,
                'initial_epoch':0,
                'steps_per_epoch':None,
                'validation_steps':None}

model.fit(X,
          y,
          epochs = model_params['epochs'],
           batch_size = model_params['batch_size'],
           #callbacks= model_params['callbacks'],
           verbose = model_params['verbose'],
           validation_split = model_params['validation_split'],
           validation_data = model_params['validation_data'],
           shuffle = model_params['shuffle'],
           initial_epoch = model_params['initial_epoch'],
           steps_per_epoch = model_params['steps_per_epoch'],
           validation_steps = model_params['validation_steps'])

Epoch 1/20
556/556 [==============================] - 20s 33ms/step - loss: 2.8337 - val_loss: 2.7388
Epoch 2/20
556/556 [==============================] - 18s 32ms/step - loss: 2.7063 - val_loss: 2.6747
Epoch 3/20
556/556 [==============================] - 18s 32ms/step - loss: 2.6364 - val_loss: 2.6126
Epoch 4/20
556/556 [==============================] - 18s 32ms/step - loss: 2.5828 - val_loss: 2.5761
Epoch 5/20
556/556 [==============================] - 18s 32ms/step - loss: 2.5455 - val_loss: 2.5546
Epoch 6/20
556/556 [==============================] - 18s 32ms/step - loss: 2.5138 - val_loss: 2.5384
Epoch 7/20
556/556 [==============================] - 18s 32ms/step - loss: 2.4904 - val_loss: 2.5230
Epoch 8/20
556/556 [==============================] - 18s 32ms/step - loss: 2.4660 - val_loss: 2.5144
Epoch 9/20
556/556 [==============================] - 23s 42ms/step - loss: 2.4463 - val_loss: 2.5127
Epoch 10/20
556/556 [==============================] - 18s 33ms/step - loss: 2.418

In [48]:
# The function to generate text from model
def Lyrics_Generator(starter,Ch_count): #,temperature=1.0):
    generated= ""
    starter = starter 
    seed=[chars_int[char] for char in starter]
    generated += starter 
    # Generating new text of given length
    for i in range(Ch_count):
        seed=[chars_int[char] for char in starter]
        x_pred = np.reshape(seed, (1, len(seed), 1))
        x_pred = x_pred/ float(n_vocab)
        prediction = model.predict(x_pred, verbose=0)[0]  
        # Getting the index of the next most probable index
        prediction = np.asarray(prediction).astype('float64')
        prediction = np.log(prediction) / 1.0 
        exp_preds = np.exp(prediction)
        prediction = exp_preds / np.sum(exp_preds)
        probas = np.random.multinomial(1, prediction, 1)
        index = np.argmax(prediction)
        next_char = int_chars[index]  
        # Generating new text
        generated += next_char
        starter = starter[1:] + next_char
       
    return generated

In [51]:
#Generating a verse from the model
verse_1 = Lyrics_Generator("my island home", 100)
#Let's have a look at the verse
verse_1

'my island home th ther weacan wo hane toe ther we can to tee te cane toe ther we can to tee te cane toe ther we ca'

In [45]:
''' Failed Method 2 of generating lyrics '''
# # set a random seed :
# # start = np.random.randint(0, len(data_X)-1)
# # pattern = data_X[start]
# # print('Seed : ')
# # print("\"",''.join([int_chars[value] for value in pattern]), "\"\n")
# # print(len(pattern))
# '''
# The default method is to use a seed to generate starting sentence.
# For greater flexibility, I implemented the below code so that we can manually include our input word/sentence
# '''

# input = 'singapore is my island home'
# char_list = [char for char in input]
# pattern = []

# for c in char_list:
#   for k, v in int_chars.items():
#     if c == v:
#       pattern.append(k)


# # How many characters you want to generate
# # generated_characters = 100
# generated_characters = len(pattern)

# # Generate Characters :
# for i in range(generated_characters):
#     x = np.reshape(pattern, ( 1, len(pattern), 1))
#     x = x / float(n_vocab)
#     prediction = model.predict(x,verbose = 0)
#     index = np.argmax(prediction)
#     result = int_chars[index]
#     #seq_in = [int_chars[value] for value in pattern]
#     sys.stdout.write(result)
#     pattern.append(index)
#     pattern = pattern[1:len(pattern)]
# print('\n\nDone')

 th the sear we can to hane

Done


Generating lyrics for CHORUS

In [53]:
chorus_df = pd.DataFrame(
    {'chorus': chorus_list})

chorus_df['chorus'] = chorus_df['chorus'].str.join(', ')
chorus_df.head()

,chorus
0,"So will you swim the current, Will you scale new heights, Will you make it happen, Will you let your dreams take flight, And will you make the difference, Will you seize the day, Will you live each moment, Will you dare to find new ways"
1,"So will you swim the current, Will you scale new heights, Will you make it happen, Will you let your dreams take flight, And will you make the difference, Will you seize the day, Will you live each moment, Will you dare to find new ways"
2,"Just my life story, Minute by second a story, That goes on forever with each breath that I take, This is my life story, Uneventfullest story, That ages with each year and birthday cake"
3,"Just my life story, Minute by second a story, That goes on forever with each breath that I take, This is my life story, Uneventfullest story, That ages with each year and birthday cake"
4,"This is where we'll see, Familiar faces from our neighbourhood, This is where we'll share, The stories dating back to our childhood, It's here where I'll find, Your hand to lift me up, When I fall, It's here where I am sure, I'm safe, I won't be lost"


In [54]:
"""
Remove punctuations, decontractions, emojis and URLs
"""

chorus_df['chorus'] = chorus_df.apply(lambda row : U.remove_punct(row['chorus']), axis=1)
chorus_df['chorus'] = chorus_df.apply(lambda row : U.remove_url(row['chorus']), axis=1)
chorus_df['chorus'] = chorus_df.apply(lambda row : U.remove_emoji(row['chorus']), axis=1)
chorus_df['chorus'] = chorus_df.apply(lambda row : U.decontraction(row['chorus']), axis=1)
chorus_df['chorus']=  chorus_df.apply(lambda row : row['chorus'].lower(), axis=1)
chorus_df.head()

,chorus
0,so will you swim the current will you scale new heights will you make it happen will you let your dreams take flight and will you make the difference will you seize the day will you live each moment will you dare to find new ways
1,so will you swim the current will you scale new heights will you make it happen will you let your dreams take flight and will you make the difference will you seize the day will you live each moment will you dare to find new ways
2,just my life story minute by second a story that goes on forever with each breath that i take this is my life story uneventfullest story that ages with each year and birthday cake
3,just my life story minute by second a story that goes on forever with each breath that i take this is my life story uneventfullest story that ages with each year and birthday cake
4,this is where well see familiar faces from our neighbourhood this is where well share the stories dating back to our childhood its here where ill find your hand to lift me up when i fall its here where i am sure im safe i wont be lost


In [55]:
choruses = chorus_df['chorus'].tolist()

In [56]:
choruses[:10]

['so will you swim the current will you scale new heights will you make it happen will you let your dreams take flight and will you make the difference will you seize the day will you live each moment will you dare to find new ways',
 'so will you swim the current will you scale new heights will you make it happen will you let your dreams take flight and will you make the difference will you seize the day will you live each moment will you dare to find new ways',
 'just my life story minute by second a story that goes on forever with each breath that i take this is my life story uneventfullest story that ages with each year and birthday cake',
 'just my life story minute by second a story that goes on forever with each breath that i take this is my life story uneventfullest story that ages with each year and birthday cake',
 'this is where well see familiar faces from our neighbourhood this is where well share the stories dating back to our childhood its here where ill find your hand t

In [57]:
# Save verses in .txt file
with open('choruses.txt', 'w',encoding="utf-8") as filehandle:  
    for listitem in choruses:
        filehandle.write('%s\n' % listitem)

In [58]:
textFileName = 'choruses.txt'
raw_text_chorus = open(textFileName, encoding = 'UTF-8').read()

In [66]:
# Mapping chars to ints
#Make two dictionaries: one to convert chars to ints , the other to convert ints back to chars
chars = sorted(list(set(raw_text_chorus)))
int_chars = dict((i, c) for i, c in enumerate(chars))
chars_int = dict((c, i) for i, c in enumerate(chars))

# int_chars = dict((i, c) for i, c in enumerate(chars))
# chars_int = dict((i, c) for c, i in enumerate(chars))

In [67]:
# Get number of chars and vocab in our text :
n_chars = len(raw_text_chorus)
n_vocab = len(chars)

In [68]:
print('Total Characters : ' , n_chars) # number of all the characters in verses.txt
print('Total Vocab : ', n_vocab) # number of unique characters

Total Characters :  26142
Total Vocab :  27


In [70]:
#make samples and labels
# process the dataset:
seq_len = 100
data_X = []
data_y = []

for i in range(0, n_chars - seq_len, 1):
    # Input Sequeance(will be used as samples)
    seq_in  = raw_text_chorus[i:i+seq_len]
    # Output sequence (will be used as target)
    seq_out = raw_text_chorus[i + seq_len]
    # Store samples in data_X
    data_X.append([chars_int[char] for char in seq_in])
    # Store targets in data_y
    data_y.append(chars_int[seq_out])
n_patterns = len(data_X)
print( 'Total Patterns : ', n_patterns)

Total Patterns :  26042


In [71]:
"""
prepare the samples and labels to be ready to go into our model.

- Reshape the samples
- Normalize them
- One hot encode the output targets
"""

# Reshape X to be suitable to go into RNN :
X = np.reshape(data_X , (n_patterns, seq_len, 1))
# Normalizing input data :
X = X/ float(n_vocab)
# One hot encode the output targets :
y = np_utils.to_categorical(data_y)

Build and Train RNN Model

In [72]:
model_chorus = Sequential()

#add output layer
model_chorus.add(SimpleRNN(units=128, input_shape=(X.shape[1], X.shape[2]), activation="relu"))
model_chorus.add(Dense(y.shape[1]))
model_chorus.add(Dropout(0.2, input_shape=(X.shape[1], X.shape[2])))
model_chorus.add(Flatten())
model_chorus.add(Dense(y.shape[1]))
model_chorus.add(Activation('softmax'))
model_chorus.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

In [73]:
model_chorus.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (None, 128)               16640     
                                                                 
 dense_2 (Dense)             (None, 27)                3483      
                                                                 
 dropout_1 (Dropout)         (None, 27)                0         
                                                                 
 flatten_1 (Flatten)         (None, 27)                0         
                                                                 
 dense_3 (Dense)             (None, 27)                756       
                                                                 
 activation_1 (Activation)   (None, 27)                0         
                                                                 
Total params: 20,879
Trainable params: 20,879
Non-trai

In [74]:
# Fit the model :
model_params = {'epochs':20,
                'batch_size':32,
                #'callbacks':callbacks_list,
                'verbose':1,
                'validation_split':0.2,
                'validation_data':None,
                'shuffle': True,
                'initial_epoch':0,
                'steps_per_epoch':None,
                'validation_steps':None}

model_chorus.fit(X,
          y,
          epochs = model_params['epochs'],
           batch_size = model_params['batch_size'],
           #callbacks= model_params['callbacks'],
           verbose = model_params['verbose'],
           validation_split = model_params['validation_split'],
           validation_data = model_params['validation_data'],
           shuffle = model_params['shuffle'],
           initial_epoch = model_params['initial_epoch'],
           steps_per_epoch = model_params['steps_per_epoch'],
           validation_steps = model_params['validation_steps'])

Epoch 1/20
652/652 [==============================] - 26s 38ms/step - loss: 2.7692 - val_loss: 2.6756
Epoch 2/20
652/652 [==============================] - 25s 39ms/step - loss: 2.5562 - val_loss: 2.6030
Epoch 3/20
652/652 [==============================] - 21s 32ms/step - loss: 2.4315 - val_loss: 2.5747
Epoch 4/20
652/652 [==============================] - 21s 32ms/step - loss: 2.3438 - val_loss: 2.5665
Epoch 5/20
652/652 [==============================] - 20s 31ms/step - loss: 2.2565 - val_loss: 2.5157
Epoch 6/20
652/652 [==============================] - 21s 32ms/step - loss: 2.1882 - val_loss: 2.5100
Epoch 7/20
652/652 [==============================] - 22s 33ms/step - loss: 2.1154 - val_loss: 2.5563
Epoch 8/20
652/652 [==============================] - 26s 40ms/step - loss: 2.0528 - val_loss: 2.4941
Epoch 9/20
652/652 [==============================] - 21s 33ms/step - loss: 1.9914 - val_loss: 2.5202
Epoch 10/20
652/652 [==============================] - 21s 32ms/step - loss: 1.940

In [79]:
chorus_1 = Lyrics_Generator("the singaporean dream", 100)
#Let's have a look at the verse
chorus_1

'the singaporean dreame te the se can tou tee tor ther we can to hane the tene the sene the ther whar we can to tee se te '